In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import torch
import os
import torchmetrics
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn as nn
from PIL import Image
from torchvision import transforms
from tqdm.notebook import tqdm_notebook as tqdm

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
# class_sample_count = [100,1]
# weights = 1 / torch.Tensor(class_sample_count)
# weights = weights.double()
# sampler = torch.utils.data.sampler.WeightedRandomSampler(
# weights=weights,
# num_samples=100000,
# replacement=True)

In [5]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import h5py,io
import albumentations as A
from albumentations.pytorch import ToTensorV2

class HDF5Dataset(Dataset): #defining the dataset 
    def __init__(self, data, metadata, transform=None):
        self.data = h5py.File(data, 'r') #creating data argument
        if type(metadata) is pd.DataFrame:
            self.metadata = metadata
        else:
            self.metadata = pd.read_csv(metadata)
        self.transform = transform #creating transform argument 

    def __len__(self):
        return len(self.metadata) #returns dataset 

    def __getitem__(self, idx):#gets image and label 
        img_name = self.metadata.isic_id[idx] #accesses image filepath 
        image = np.array(self.data[img_name]) #opens image
        image = np.array(Image.open(io.BytesIO(image)),dtype=np.float32)/255

        label = int(self.metadata['target'].iloc[idx]) #find the label 
        
        if self.transform:
            augmented = self.transform(image=image) #transformation 
            image = augmented['image'] #grab the augmented image 

        return image, label #if you want to include metadata we would need to return more fields here   
dataset = '/kaggle/input/isic-2024-challenge/train-image.hdf5'
train_data = '/kaggle/input/isic-2024-challenge/train-metadata.csv'
# Train Transform can be tailored according to the needs. Or it can None
train_transform = A.Compose([
    A.Resize(height=224, width=224), #resize 
    A.OneOf([A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15),
             A.RandomBrightnessContrast() 
             ], p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    # !!One needs to change the mean and std values to appropriate ones for this dataset.!!
    A.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=1.0),
    ToTensorV2(),
])

#calls the custom dataset to access training data 
train_dataset = HDF5Dataset(dataset, train_data, transform=train_transform) 
#load training data 
train_loader = DataLoader(train_dataset,
                          batch_size=32,
                          shuffle=True,
#                           sampler = sampler,
                          num_workers=4) 

/tmp/ipykernel_34/1763307054.py:13: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  self.metadata = pd.read_csv(metadata)


In [ ]:
import torch
import torch.nn as nn
from torchvision.models import efficientnet_b0

eff = efficientnet_b0(pretrained = True)

num_features = eff.classifier[1].in_features
eff.classifier = nn.Sequential(
#     nn.Dropout(p=0.5, inplace=True),
    nn.Linear(num_features, 512),
    nn.ReLU(),
#     nn.Dropout(p=0.5, inplace=True),
    nn.Linear(512, 128),
    nn.ReLU(),
#     nn.Dropout(p=0.5, inplace=True),
    nn.Linear(128, 1),
    nn.Sigmoid()
)

print(num_features)

for param in eff.parameters():
    param.requires_grad = True

for param in eff.features.parameters():
    param.requires_grad = False

eff = eff.to(device)

In [ ]:
lossfx = nn.BCELoss()
lossfx.to(device)
optimizer = torch.optim.Adam(eff.parameters(), lr = 0.0005)

In [ ]:
Epoch = 5
for epoch in tqdm(range(Epoch)):
    net_loss = 0.0
    eff.train()
    for image, label in tqdm(train_loader):
#         x = data['image'].to(device)
#         y = data['label'].to(device)
        x = image.to(device)
        y = label.to(device)
        pred = eff(x)
        loss_ = lossfx(pred.squeeze(), y.type(torch.float))
#         net_loss += loss_.item()
        del x
        del y
        optimizer.zero_grad()
        loss_.backward()
        optimizer.step()
        torch.cuda.empty_cache()
    print(f"Epoch {epoch+1}")
#     mdl.eval()
#     with torch.inference_mode():
#         for data in tqdm(val_load):
#             x = data['image'].to(device)
#             y = data['label'].to(device)
#             pred = mdl(x)
#             acc.update(pred, y)
#         print(f"Validation Accuracy - {acc.compute()}")

In [ ]:
torch.save({'model_state_dict':eff.state_dict(),
            'arch':'efficientnet_b0'},
            'eff_b0_3.pth')